In [ ]:
### uses sheet 1 from scb and  sheets antal avlidna och  åldersgrupp from fhm 

#fhm_age_grp = pd.read_csv('fhm_dead_age_grp.csv',sep=';')

#fhm_dead = pd.read_csv('fhm_scb_dead.csv',sep=';',header=0,index_col=0,parse_dates=True,names=['fhm_dead'])

## NOTE! - assuming population 2020 grows as it did 2918-2019
## other prg's use mean of two consequtive years, or pop year before
## therefore this shows slightly different than eg scb_population

#### !!!!! ANOMALY in SCB data: historical death data for (at least ) 2019 from statistikdatabasen 
# http://www.statistikdatabasen.scb.se/pxweb/sv/ssd/START__BE__BE0101__BE0101G/ManadFoddDod/table/tableViewLayout1/
# does not concur with death data 2019 in the preliminary data! 

### DUE to okänd dödsdag when using the daily data of the preliminary stuff - much more okänd dag than okänd månad in 
### the monthly stuff
####

# ALL DATA BASED ON SCB PREL DATA IGNORES THOSE WITH UNKNOWN DAY OF DEATH !!!

#### BEFORE USING, check that scb and fhm have about same number of days of data

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests

sns.set()


In [ ]:
def parse_scb_prel():
    
    def monthname2number(m):
        if m == 'januari' : return 1
        if m == 'februari' : return 2
        if m == 'mars' : return 3
        if m == 'april' : return 4
        if m == 'maj' : return 5
        if m == 'juni' : return 6
        if m == 'juli' : return 7
        if m == 'augusti' : return 8
        if m == 'september' : return 9
        if m == 'oktober' : return 10
        if m == 'november' : return 11
        if m == 'december' : return 12

    df = pd.read_csv('scb-dead_per_day.csv',sep=';',usecols=[1,2,3,4,5,6,7,10,11],thousands=',')
    
    df['month'] = df['Månad'].apply(monthname2number)
    df[['Månad','month']] = df[['Månad','month']].replace(np.nan,0)
    df['month'] = df['month'].astype(int)
    
    df = df.replace(np.nan,0)

    
    df[['2015','2016','2017','2018','2019','2020','2015-2019']] = \
    df[['2015','2016','2017','2018','2019','2020','2015-2019']].astype(int)

    
    unknown_death_day = df.iloc[-1,:7]
    
    df.drop(366,inplace=True)
    
    df = df.loc[df['2020'] != 0]
    
    #df = df.drop('Månad',axis=1)
    
    index = pd.date_range('2020-01-01',periods=len(df))
    df.index = index
    
    known_death_day = df.iloc[:,:7].sum()
    
    year_fraction = len(df) / 366
    
    unknown_death_day[['2015','2016','2017','2018','2019','2015-2019']] = unknown_death_day[
    ['2015','2016','2017','2018','2019','2015-2019']] * year_fraction
    
    dead_pct_per_day = df[['2015','2016','2017','2018','2019','2020','2015-2019']] / known_death_day
    
    all_deaths = df[['2015','2016','2017','2018','2019','2020','2015-2019']] + unknown_death_day * dead_pct_per_day
    
    all_deaths = pd.concat([all_deaths,df[['Månad','Dag']]],axis=1)
    
    return all_deaths

In [ ]:
scb_deaths = parse_scb_prel()
scb_deaths.rename(columns={'Dag' :'day'},inplace=True)
scb_deaths

In [ ]:
def translate_month(m):
    if m == 'januari' : return  'January'
    if m == 'februari' : return 'February'
    if m == 'mars' : return 'March'
    if m == 'april' : return 'April'
    if m == 'maj' : return 'May'
    if m == 'juni' : return 'June'
    if m == 'juli' : return 'July'
    if m == 'augusti' : return 'August'
    if m == 'september' : return 'September'
    if m == 'oktober' : return 'October'
    if m == 'november' : return 'November'
    if m == 'december' : return 'December'
    
    
scb_deaths['month'] = scb_deaths['Månad'].apply(translate_month)

scb_deaths = scb_deaths.drop('Månad',axis=1)


scb_deaths.rename(columns={'Dag' : 'day'},inplace=True)
scb_deaths['year'] = '2020'

scb_deaths['day'] = scb_deaths['day'].astype(str)
    


In [ ]:
pd.datetime.strptime('2020'+ '-'+'January'+'-'+'31','%Y-%B-%d')

In [ ]:
def fix_date(s):
    return pd.datetime.strptime('2020'+'-'+s.month+'-'+s.day,'%Y-%B-%d')

scb_deaths['date'] = scb_deaths.apply(lambda row : fix_date(row),axis=1)

In [ ]:
scb_deaths['date'] = pd.to_datetime(scb_deaths['date'])

In [ ]:
scb_deaths = scb_deaths.set_index('date')

In [ ]:
scb_deaths.drop(['day','month','year'],inplace=True,axis=1)

In [ ]:
last_day_mask = scb_deaths['2020'] != 0
scb_deaths = scb_deaths[last_day_mask]

scb_deaths

In [ ]:
last_day = scb_deaths.index[-1].date()
print (last_day)
last_stable_day = (last_day - pd.DateOffset(days=7)).date()
last_stable_day

In [ ]:
scb_deaths['excess'] = scb_deaths['2020'] - scb_deaths['2015-2019']
scb_deaths['cumexcess'] = scb_deaths['excess'].cumsum()
scb_deaths['excess_19'] = scb_deaths['2019'] - scb_deaths['2015-2019']
scb_deaths['cumexcess_19'] = scb_deaths['excess_19'].cumsum()
scb_deaths_orig = scb_deaths.copy()
scb_deaths

In [ ]:
scb_deaths.sum()['2019'] - scb_deaths.sum()['2015-2019']

In [ ]:
scb_deaths = scb_deaths.loc[:last_stable_day].copy()
scb_deaths

In [ ]:
scb_deaths.sum()

In [ ]:

plt.figure(figsize=(18,12))
plt.title('SWEDEN Daily Excess Deaths & Cumulative Excess Deaths vs 2015-2019\n\nData inc {}\n\n'.format(last_stable_day)+\
         'DataSource : SCB.se')
ax = plt.gca()
ax2 = plt.twinx()
ax.bar(scb_deaths.index,scb_deaths['excess'],color='r',label='Excess Deaths')
ax.plot(scb_deaths.index,[0] * len(scb_deaths.index),color='k',ls='dashed')

ax2.plot(scb_deaths.index,scb_deaths['cumexcess'],color='orange',label='Cumulative Excess Deaths')
ax2.plot(scb_deaths.index,[0] * len(scb_deaths.index),color='k',ls='dashed')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')

ax.set_ylabel('Daily Excess Deaths')
ax2.set_ylabel('Cumulative Excess Deaths')
plt.savefig('fhm_scb_excess.jpg',format='jpg')

In [ ]:
### population 2015-2019
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'
data = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:ÅlderTotA",
        "values": []
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2014",  
          "2015",
          "2016",
          "2017",
          "2018",
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url,json=data)
r.status_code

In [ ]:
json = r.json()

In [ ]:
json['data'][0]

In [ ]:
data_list = []

for rec in range(len(json['data'])):
    data_list.append((json['data'][rec]['key'][0],json['data'][rec]['key'][1],json['data'][rec]['values'][0]))

In [ ]:
population = pd.DataFrame(data_list,columns=['gender','year','population'])
population = population.set_index('year')
population['population'] = population['population'].astype(int)

In [ ]:
population

In [ ]:
population = population.groupby(population.index).sum()
population['growth'] = population['population'] / population['population'].shift()


population.loc['2020','population'] = population.loc['2019','population']  
population['population'] = population['population'].astype(int)
population

In [ ]:
scb_deaths_ytd = scb_deaths.sum()
scb_deaths_ytd = scb_deaths_ytd.drop(['2015-2019','excess','cumexcess'])
scb_deaths_ytd

In [ ]:
deaths_per_M_ytd = scb_deaths_ytd / (population['population'] / 1e6)
deaths_per_M_ytd.drop('2014',inplace=True)
deaths_per_M_ytd

In [ ]:
scb_deaths

In [ ]:
scb_deaths.loc['2020-01-01' : '2020-08-31','2019'].sum()

In [ ]:
population

In [ ]:
scb_deaths.loc['2020-01-01' : '2020-08-31','2019'].sum()

In [ ]:
scb_deaths.loc['2020-01-01' : '2020-08-31','2019'].sum() / population.loc['2019','population']

In [ ]:
maximums = pd.concat ([scb_deaths.max(),
            pd.Series(scb_deaths.index[scb_deaths.apply(np.argmax)].values,
                      index=scb_deaths.max().index)],axis=1)
maximums.columns = ['max','date_of_max']
maximums

In [ ]:
minimums = pd.concat ([scb_deaths.min(),
            pd.Series(scb_deaths.index[scb_deaths.apply(np.argmin)].values,
                      index=scb_deaths.min().index)],axis=1)
minimums.columns = ['min','date_of_min']
minimums

In [ ]:
daily_deaths_per_M = scb_deaths.loc[:,'2015' : '2020'].div(population['population'] / 1e6,axis=1)
daily_deaths_per_M.drop('2014',axis=1,inplace=True)
daily_deaths_per_M 

In [ ]:
daily_deaths_per_M.sum()

In [ ]:
daily_deaths_per_M.sum().plot(kind='bar',
                              figsize=(18,12),
                              color='r',
                              title='Sweden Deaths per Million YTD {}\n\nDataSource :SCB.se'.format(last_stable_day))
plt.ylabel('deaths per million')
plt.savefig('fhm_scb_deaths_per_M.jpg',format='jpg')

In [ ]:
daily_deaths_per_M_2015_2019_avg = daily_deaths_per_M[['2015','2016','2017','2018','2019']].mean(axis=1)
daily_deaths_per_M_2015_2019_avg

In [ ]:
plt.figure(figsize=(18,12))
plt.plot(daily_deaths_per_M_2015_2019_avg)
plt.plot(daily_deaths_per_M['2020'])
plt.title('Sweden deaths per day per million YTD, 2020 vs average 2015-2019')
plt.ylabel('deaths per million per day')

In [ ]:
import calendar
d = dict(enumerate(calendar.month_abbr))

In [ ]:
monthly_deaths_per_M = daily_deaths_per_M.groupby(daily_deaths_per_M.index.month).sum()
month_names = pd.Series(monthly_deaths_per_M.index.values).map(d)
month_names
monthly_deaths_per_M.index = month_names
monthly_deaths_per_M.index.name = 'month'
monthly_deaths_per_M

In [ ]:
monthly_deaths_per_M.sum()

In [ ]:
ytd_deaths_per_M_avg = monthly_deaths_per_M.loc[:,:'2019'].sum().mean()
ytd_deaths_per_M_avg

In [ ]:
monthly_deaths_per_M.T.plot(kind='bar',
                            stacked=True,figsize=(18,12),
                           title='Sweden deaths per million YTD {}, monthly cumulative\n\nDataSource:SCB.se'.format(
                               last_stable_day))
plt.ylabel('deaths per million')
plt.axhline(ytd_deaths_per_M_avg,ls='dashed',color='orange',label='2015-2019 average')
plt.legend(loc='lower center')

plt.savefig('fhm_deaths_per_M_stacked_monthly_YTD.jpg',format='jpg')

In [ ]:
# FHM excel sheed döda åldersgrupp
fhm_age_grp = pd.read_csv('fhm_dead_age_grp.csv',sep=';')
age_grp = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90+']
fhm_age_grp.drop('Åldersgrupp',axis=1,inplace=True)
fhm_cols = ['cases','icu','dead']
fhm_age_grp.columns = fhm_cols
fhm_age_grp['age_grp'] = age_grp
fhm_age_grp.set_index('age_grp',inplace=True,drop=True)
fhm_age_grp

In [ ]:
print (fhm_age_grp.sum())
scb_deaths['2020'].sum()

In [ ]:
fhm_age_grp['dead'].cumsum().plot(style='ro--',
                                  title='SWEDEN Covid-19 deaths per age group, cumulative,Y2D {}\n\nDataSource : SCB,FHM'.format(last_day),
                                  figsize=(18,12))
plt.ylabel('total number of deaths')
_= plt.xticks(range(len(fhm_age_grp)),age_grp,rotation=90)
plt.savefig('fhm_scb_cumulative_covid_deaths.jpg',format='jpg')

In [ ]:
# population stratified

url2 = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'

data2 = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årA",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r2 = requests.post(url2,json=data2)
r2.status_code

In [ ]:
json2 = r2.json()
json2['data'][0]

In [ ]:
data2_list = []

for rec2 in range(len(json2['data'])):
    
    data2_list.append((json2['data'][rec2]['key'][0],json2['data'][rec2]['key'][1],json2['data'][rec2]['key'][2],
                      json2['data'][rec2]['values'][0]))

In [ ]:
population_age_grp = pd.DataFrame(data=data2_list,columns=['age_grp','gender','year','pop'])
population_age_grp['pop'] = population_age_grp['pop'].astype(int)
population_age_grp['age_grp'] = population_age_grp['age_grp'].apply(lambda s : s.replace('+',''))
population_age_grp['age_grp'] = population_age_grp['age_grp'].astype(int)
population_age_grp

In [ ]:
population_age_grp = pd.pivot_table(population_age_grp,index='age_grp',columns='gender',values='pop')
population_age_grp

In [ ]:
population_age_grp['pop'] = population_age_grp.sum(axis=1)
population_age_grp.drop(['1','2'],axis=1,inplace=True)


In [ ]:
population_age_grp = pd.Series(population_age_grp['pop'])
population_age_grp

In [ ]:
population_age_grp.loc['30':'39'].sum()

In [ ]:
population_age_grp

In [ ]:
zero_9 = pd.Series(population_age_grp.loc[0:9].sum())
ten_19 = pd.Series(population_age_grp.loc[10:19].sum())
twenty_29 = pd.Series(population_age_grp.loc[20:29].sum())
thirty_39 = pd.Series(population_age_grp.loc[30:39].sum())
forty_49 = pd.Series(population_age_grp.loc[40:49].sum())
fifty_59 = pd.Series(population_age_grp.loc[50:59].sum())
sixty_69 = pd.Series(population_age_grp.loc[60:69].sum())
seventy_79 = pd.Series(population_age_grp.loc[70:79].sum())
eighty_89 = pd.Series(population_age_grp.loc[80:89].sum())
ninety_plus = pd.Series(population_age_grp.loc[90:].sum())

thirty_39

In [ ]:
population_age_grp = pd.DataFrame([zero_9,ten_19,twenty_29,thirty_39,forty_49,fifty_59,
                               sixty_69,seventy_79,eighty_89,ninety_plus],index=age_grp)

population_age_grp.rename(columns={0:'pop'},inplace=True)

population_age_grp

In [ ]:
population_age_grp['pop'].sum()

In [ ]:
population_age_grp

In [ ]:
fhm_age_grp

In [ ]:
covid_dead_per_age_grp = fhm_age_grp['dead'] / population_age_grp['pop']
covid_dead_per_age_grp

In [ ]:
(100 * covid_dead_per_age_grp).plot(kind='bar',color='r',figsize=(18,12),
                                    title='SWEDEN percent COVID-19 dead of pop. per age group, Y2D {}\n\nDataSource : SCB, FHM'.format(last_day))
plt.ylabel('percent of age group')
#plt.yscale('log')

In [ ]:
fhm_age_grp.plot(kind='bar',
                 figsize=(18,12),
                 color=['green','orange','red'],
                title='SWEDEN Nr. Cases, ICUs, Deaths per Age Group YTD {}\n\nDataSource : SCB,FHM'.format(last_day))
plt.ylabel('number of persons')
plt.savefig('fhm_case_icu_deaths.jpg',format='jpg')

In [ ]:
fhm_data_per_age_group_capita = fhm_age_grp.div(population_age_grp['pop'],axis=0)
fhm_data_per_age_group_capita

In [ ]:
(100 * fhm_data_per_age_group_capita).plot(kind='bar',title='SWEDEN percent of age group impacted Y2D {} by COVID-19\n\nDataSource : SCB, FHM'.format(last_day),
                                           figsize=(18,12),color=['green','orange','red'])
plt.ylabel('percent of age group')
plt.savefig('fhm_pct_age_grp_impacted.jpg',format='jpg')

In [ ]:
scb_deaths = scb_deaths_orig # include all dates
scb_deaths


In [ ]:
fhm_dead = pd.read_csv('fhm_scb_dead.csv',sep=';',header=0,index_col=0,parse_dates=True,names=['fhm_dead'])
fhm_dead

In [ ]:
cmp_deaths = fhm_dead.join(scb_deaths['2020'])


In [ ]:
cmp_deaths.dropna(inplace=True)

In [ ]:
cmp_deaths['pct_covid'] = cmp_deaths['fhm_dead'] / cmp_deaths['2020']
cmp_deaths['non_covid'] = cmp_deaths['2020'] - cmp_deaths['fhm_dead']
cmp_deaths.columns=['covid_dead','all_dead','pct_covid','non_covid_dead']
cmp_deaths['pct_non_covid'] = cmp_deaths['non_covid_dead'] / cmp_deaths['all_dead']
cmp_deaths

In [ ]:
cmp_deaths = cmp_deaths.astype(float)

In [ ]:
covid_death_pct_in_period = cmp_deaths.sum()['covid_dead'] / cmp_deaths.sum()['all_dead']
covid_death_pct_in_period

In [ ]:
covid_death_pct_YTD = cmp_deaths.sum()['covid_dead'] / scb_deaths['2020'].sum()
covid_death_pct_YTD

In [ ]:
days_of_year = pd.to_datetime(last_stable_day).dayofyear
year_fraction = days_of_year / 365

pred_covid_deaths = cmp_deaths.sum()['covid_dead'] * (1 / year_fraction) #worst case! 

pred_death_pct = cmp_deaths.sum()['covid_dead'] / (scb_deaths['2020'].sum() * (1/year_fraction))
pred_death_pct
pred_covid_deaths

In [ ]:
plt.figure(figsize=(18,12))
plt.bar(cmp_deaths.index.date,cmp_deaths['pct_covid'])


In [ ]:

plt.figure(figsize=(18,12))
plt.bar(cmp_deaths.index.date,cmp_deaths['non_covid_dead'],color='orange',label='non-covid')
plt.bar(cmp_deaths.index.date,cmp_deaths['covid_dead'],bottom=cmp_deaths['non_covid_dead'],
        color='crimson',label='covid')

plt.legend(loc='upper right')
plt.title('SWEDEN All Daily Deaths 2020-03-12 - {} : Covid vs Non-Covid'.format(last_stable_day))
plt.ylabel('nr of daily deaths')
plt.savefig('fhm_covid_vs_non_covid_deaths_abs.jpg',format='jpg')

In [ ]:
plt.figure(figsize=(18,12))
plt.bar(cmp_deaths.index.date,cmp_deaths['pct_non_covid'],color='orange',label='non-covid')
plt.bar(cmp_deaths.index.date,cmp_deaths['pct_covid'],bottom=cmp_deaths['pct_non_covid'],
        color='crimson',label='covid')

plt.legend(loc='upper left')
plt.title('SWEDEN All Daily Deaths 2020-03-12 - {} : Covid vs Non-Covid, ratio of All deaths'.format(last_stable_day))
plt.ylabel('ratio of all daily deaths')
plt.savefig('fhm_covid_vs_non_covid_deaths_rel.jpg',format='jpg')

In [ ]:
first_day = pd.to_datetime("2020-03-12")
first_day.day_name()

In [ ]:
weekly_scb = scb_deaths.loc[:,'2015' : '2020'].resample('W-TUE').sum()
weekly_scb

In [ ]:
weekly_fhm = fhm_dead.resample('W-TUE').sum()
weekly_fhm

In [ ]:
weekly_deaths = weekly_scb.join(weekly_fhm,how='outer')
weekly_deaths.rename(columns={'fhm_dead' : 'covid_dead'},inplace=True)
weekly_deaths['covid_dead'].fillna(0,inplace=True)
weekly_deaths

In [ ]:
weekly_deaths['2015-2018'] = weekly_deaths.loc[:,'2015' : '2018'].mean(axis=1)
weekly_deaths['non_covid_dead'] = weekly_deaths['2020'] - weekly_deaths['covid_dead']
weekly_deaths.index = weekly_deaths.index.date
weekly_deaths['max'] = weekly_deaths.loc[:,'2015' :'2018'].max(axis=1)
weekly_deaths['min'] = weekly_deaths.loc[:,'2015' :'2018'].min(axis=1)
weekly_deaths = weekly_deaths[:-1]
weekly_deaths

In [ ]:
plt.figure(figsize=(18,12))
plt.title('SWEDEN weekly Covid vs Non-Covid deaths 2020 cmp average 2015-2018')
plt.bar(weekly_deaths.index,weekly_deaths['non_covid_dead'],label='non_covid_deaths',color='orange')
plt.bar(weekly_deaths.index,weekly_deaths['covid_dead'],bottom=weekly_deaths['non_covid_dead'],
       label='covid_deaths',color='red')
plt.plot(weekly_deaths.index,weekly_deaths['2015-2018'],'o--',color='k',label='mean 2015-2018')

plt.fill_between(weekly_deaths.index,weekly_deaths['min'],weekly_deaths['max'],color='c',alpha=0.3,
                label='range 2015-2018')

plt.legend(loc='upper left')
plt.ylabel('number of deaths per week')

In [ ]:
population

In [ ]:
# pop from end of previous year
pop = population.shift()
pop = pop['population']
pop['covid_dead'] = pop['2020']
pop.dropna(inplace=True)
pop

In [ ]:
weekly_pop_adjusted_deaths = weekly_deaths.loc[:,'2015' : 'covid_dead'].copy()
weekly_pop_adjusted_deaths = weekly_pop_adjusted_deaths / (pop / 1e6)
weekly_pop_adjusted_deaths['2015-2018'] = weekly_pop_adjusted_deaths.loc[:,'2015' : '2018'].mean(axis=1)
weekly_pop_adjusted_deaths['min'] = weekly_pop_adjusted_deaths.loc[:,'2015' : '2018'].min(axis=1)
weekly_pop_adjusted_deaths['max'] = weekly_pop_adjusted_deaths.loc[:,'2015' : '2018'].max(axis=1)

weekly_pop_adjusted_deaths['non_covid_dead'] = weekly_pop_adjusted_deaths['2020'] - weekly_pop_adjusted_deaths['covid_dead']
weekly_pop_adjusted_deaths['week_idx'] = range(len(weekly_pop_adjusted_deaths))
weekly_pop_adjusted_deaths

In [ ]:
excess = weekly_pop_adjusted_deaths['2020'] - weekly_pop_adjusted_deaths['2015-2018']
excess

In [ ]:
cum_excess = excess.cumsum()
cum_excess

In [ ]:
cum_excess * (pop['2020'] / 1e6)

In [ ]:
last_week = cum_excess.index[-1]

In [ ]:

ax = excess.plot(figsize=(18,12),style='ro--',label='weekly excess deaths',
                title='SWEDEN weekly excess deaths & cumulative excess deaths, per million')
ax2 = plt.twinx()

cum_excess.plot(ax=ax2,style='yo--',label='cumulative excess deaths')

ax.legend(loc='upper left')
ax2.legend(loc='upper right')

ax.set_ylabel('excess deaths per million')
ax2.set_ylabel('cumulative excess deaths per million')

In [ ]:
plt.figure(figsize=(18,12))
plt.title('SWEDEN weekly Covid vs Non-Covid deaths per Million, YTD {}, 2020 cmp average 2015-2018'.format(last_week))
plt.bar(weekly_pop_adjusted_deaths.index,weekly_pop_adjusted_deaths['non_covid_dead'],label='non_covid_deaths',color='orange')
plt.bar(weekly_pop_adjusted_deaths.index,weekly_pop_adjusted_deaths['covid_dead'],bottom=weekly_pop_adjusted_deaths['non_covid_dead'],
       label='covid_deaths',color='red')
plt.plot(weekly_pop_adjusted_deaths.index,weekly_pop_adjusted_deaths['2015-2018'],'o--',color='k',label='mean 2015-2018')

plt.fill_between(weekly_pop_adjusted_deaths.index,weekly_pop_adjusted_deaths['min'],weekly_pop_adjusted_deaths['max'],color='c',alpha=0.3,
                label='range 2015-2018')

plt.legend(loc='upper left')
plt.ylabel('weekly deaths per million ')

plt.savefig('fhm_weekly_covid_non_covid_per_m.jpg',format='jpg')

In [ ]:
weekly_pop_adjusted_deaths.sum() * (pop / 1e6) 

In [ ]:
import pymc3 as pm
import arviz as az

data = weekly_pop_adjusted_deaths.loc[:,'2015' : '2020'].copy()

data

In [ ]:
data=data.T.unstack().reset_index()
data.columns=['date','year','dead_per_M']
data

In [ ]:
unique_years = data.year.unique()
year_idx_map = dict(zip(unique_years,range(len(unique_years))))

data['year_idx'] = data['year'].apply(lambda y :year_idx_map[y])

unique_weeks = data.date.unique()
week_idx_map = dict(zip(unique_weeks,range(len(unique_weeks))))
data['week_idx'] = data['date'].apply(lambda d : week_idx_map[d])
data

In [ ]:
import pymc3 as pm
import arviz as az

year_idx = data['year_idx'].values
week_idx = data['week_idx'].values

model = pm.Model()
with model:
    
    lambda_bar = pm.Exponential('lambda_bar',1 / data['dead_per_M'].mean())
    lambda_ = pm.Exponential('lambda_',lambda_bar,shape=len(unique_weeks))
    lkh = pm.Poisson('lkh', lambda_[week_idx],observed=data['dead_per_M'])
    
    trace = pm.sample(500,tune=500)
    result = pm.trace_to_dataframe(trace)
    print (az.summary(trace,hdi_prob=0.89))
    _= az.plot_trace(trace)

In [ ]:
result = result.iloc[:,1:]
result.describe()


In [ ]:
result.columns = range(len(week_idx_map))
result.describe()

In [ ]:
CI = result.quantile([0.055,0.945],axis=0)
CI

In [ ]:
errors = np.abs(CI - result.describe().loc['mean'])
errors.loc[:,1:32]

In [ ]:
plt.figure(figsize=(18,12))
plt.title('SWEDEN Estimated weekly deaths per M vs actuals')

plt.errorbar(result.columns,result.mean(),errors.values,fmt='o--',capsize=5,color='k',
            label='Poisson Estimate inc. 2020 with 89% CI')

plt.bar(result.columns,weekly_pop_adjusted_deaths['non_covid_dead'],alpha=0.5,color='orange',
        label='non-covid-deaths')

_=plt.xticks(range(len(result.columns)),weekly_pop_adjusted_deaths.index,rotation=90)

plt.bar(result.columns,weekly_pop_adjusted_deaths['covid_dead'],alpha=0.5,color='red',
        bottom=weekly_pop_adjusted_deaths['non_covid_dead'],label='covid deaths')

plt.bar(result.columns,weekly_pop_adjusted_deaths['covid_dead'],
        bottom=weekly_pop_adjusted_deaths['non_covid_dead'],edgecolor='k',alpha=1,fill=False)

plt.fill_between(result.columns,weekly_pop_adjusted_deaths['min'],weekly_pop_adjusted_deaths['max'],
                color='c',alpha=0.5,label='2015-2018 range')

plt.plot(result.columns,weekly_pop_adjusted_deaths['2015-2018'],'o--',color='crimson',alpha=0.7,
        label='2015-2018 average')

plt.legend(loc='upper left')
plt.ylabel('deaths per million')

plt.savefig('fhm_covid_vs_non_covid_bar.jpg',format='jpg')

In [ ]:
fhm_age_grp